# Essai perceptron multi-couche sur données miniloan

## Acquisition des données d'apprentissage

In [1]:
import numpy as np

import csv;
f= open ("../miniloan_csma_10000.csv", 'r')
myReader = csv.reader(f)
f.close
X_train_origine=list()
y_train_origine=list()

for row in myReader:
    X_train_origine.append(row[2:4]+row[5:8])
    y_train_origine.append(row[8])

#X_train=np.array(X_train)
print("Nb instances = ", len(X_train_origine)," X = ", X_train_origine[0:5], " Y = ",y_train_origine[0:5])
X_train_origine=X_train_origine[1:]
y_train_origine=y_train_origine[1:]
print("Nb instances positives = ",y_train_origine.count('1'),", nb instances négatives = ",y_train_origine.count('0'))

Nb instances =  10001  X =  [['Revenu', 'Credit score', 'Montant', 'DurÃ©e', 'Taux'], ['307666', '197', '217370', '21', '0.043196103380325666'], ['416779', '947', '1015923', '7', '0.05206380610252473'], ['110310', '162', '259208', '4', '0.04068419371946227'], ['214882', '440', '267647', '13', '0.0411346949665408']]  Y =  ['Status', '0', '0', '0', '1']
Nb instances positives =  4720 , nb instances négatives =  5280


## Acquisition des données de test

In [2]:
f= open ("../miniloan_csma_1000.csv", 'r')
myReader = csv.reader(f)
f.close
X_test_origine=list()
y_test_origine=list()

for row in myReader:
    X_test_origine.append(row[2:4]+row[5:8])
    y_test_origine.append(row[8])

print("X = ", X_test_origine[0:5], " Y = ",y_test_origine[0:5])
X_test_origine=X_test_origine[1:]
y_test_origine=y_test_origine[1:]

X =  [['Revenu', 'Credit score', 'Montant', 'DurÃ©e', 'Taux'], ['502389', '150', '476355', '25', '0.04171977146573331'], ['543158', '972', '577534', '14', '0.027923264583234676'], ['464520', '928', '85594', '3', '0.04876627034264725'], ['493692', '7', '1117942', '18', '0.03615719567808266']]  Y =  ['Status', '0', '1', '1', '0']


## Normalisation des données d'apprentissage

In [3]:
from sklearn.preprocessing import StandardScaler  
scaler = StandardScaler()  
# Don't cheat - fit only on training data
scaler.fit(X_train_origine)  
X_train = scaler.transform(X_train_origine)  
# apply same transformation to test data
#X_test = scaler.transform(X_test)

print("X normalisé = ", X_train[0:5])

X normalisé =  [[ 0.0348213  -1.0387978  -1.14822718  0.62903175  0.30249088]
 [ 0.66608223  1.55517127  1.18756092 -0.99691347  0.77543804]
 [-1.10695955 -1.15984969 -1.02584995 -1.34533031  0.16852151]
 [-0.50197006 -0.19835182 -1.00116566 -0.30007981  0.1925484 ]
 [-1.16063059 -1.12872206  1.63347232  1.32586542 -0.04632843]]


## Normalisation des données de test

In [4]:
# apply same normalisation to test data

X_test = scaler.transform(X_test_origine)

print("X normalisé = ", X_test[0:5])

X normalisé =  [[ 1.16136922 -1.20135319 -0.39068938  1.09358753  0.22375268]
 [ 1.39723367  1.6416369  -0.0947382  -0.18394086 -0.51206571]
 [ 0.9422824   1.48945738 -1.53367538 -1.46146925  0.59956834]
 [ 1.11105371 -1.69593663  1.48596912  0.28061492 -0.07291992]
 [-0.21736373  0.13713484 -0.90460764 -1.34533031 -1.71571247]]


## Redéfinition de la fonction forward_pass

In [5]:
from scipy.special import expit as logistic_sigmoid
from sklearn.utils.extmath import safe_sparse_dot

def my_inplace_logistic(X):
    """Compute the logistic function inplace. From sklearn/NN/_base
    Parameters
    ----------
    X : {array-like, sparse matrix}, shape (n_samples, n_features)
        The input data.
    """
    logistic_sigmoid(X, out=X)

def my_forward_pass(mlp, instance):
        """Perform a forward pass on the network by computing the values
        of the neurons in the hidden layers and the output layer.
        Parameters
        ----------
        activations : list, length = n_layers - 1
            The ith element of the list holds the values of the ith layer.
        """
        activations=[np.array(instance)]
#        hidden_activation = ACTIVATIONS[mlp.activation]
        # Iterate over the hidden layers
        for i in range(mlp.n_layers_ - 1):
            activations.append(safe_sparse_dot(activations[i], mlp.coefs_[i]))
            activations[i + 1] += mlp.intercepts_[i]

            # For the hidden layers
            if (i + 1) != (mlp.n_layers_ - 1):
                my_inplace_logistic(activations[i + 1])
#                hidden_activation(activations[i + 1])

        # For the last layer
#        output_activation = ACTIVATIONS[mlp.out_activation_]
        my_inplace_logistic(activations[i + 1])
#        output_activation(activations[i + 1])

        return activations


## Création, paramétrage et entraînement de l'arbre de décision et du réseau sur les données d'origine et de l'arbre de décision sur les données tranformées

In [ ]:
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier

decisionTree_origine = DecisionTreeClassifier(criterion="entropy", max_depth = 20)

from sklearn.neural_network import MLPClassifier

clf = MLPClassifier(solver='lbfgs', alpha=0.0000000001, hidden_layer_sizes=(5,5,3), random_state=1, max_iter=200000, activation='logistic')
decisionTree_meta = DecisionTreeClassifier(criterion="entropy", max_depth = 10)

X_test_meta = np.empty((len(X_test),clf.hidden_layer_sizes[-1]))

out_data = np.empty((10, 10, 5))

from sklearn.utils import shuffle

for s in range(10):
    X_train_origine, X_train, y_train_origine = shuffle(X_train_origine, X_train, y_train_origine)
    print("Shuffle ",s)
    
    inst_idx =0
    for nbi in range(1000,10001,1000):
        print("Nb instances = ",nbi)

        #Construction et évaluation de l'arbre de décision
        decisionTree_origine.fit(X_train_origine[0:nbi+1],y_train_origine[0:nbi+1])
        nbRegles_origine = decisionTree_origine.get_n_leaves()
        DTscoreTest_origine = decisionTree_origine.score(X_test_origine, y_test_origine)
        print("Arbre de décision données d'origine : Nb feuilles (= règles) : ",nbRegles_origine," Score sur les données d'apprentissage = ", decisionTree_origine.score(X_train_origine[0:nbi+1],y_train_origine[0:nbi+1])," Score sur les données de test = ", DTscoreTest_origine)

        # Construction et évaluation du réseau de neurones
        clf.fit(X_train[0:nbi+1],y_train_origine[0:nbi+1])
        NNscoreTest = clf.score(X_test, y_test_origine)
        print("Réseau de neurones : Score sur les données d'apprentissage = ", clf.score(X_train[0:nbi+1],y_train_origine[0:nbi+1]),"Score sur les données de test = ", NNscoreTest)

        # Récupération de la dernière couche du réseau et transformation des données d'entrainement
        X_train_meta = np.empty((len(X_train[0:nbi+1]),clf.hidden_layer_sizes[-1]))

        for i in range(len(X_train[0:nbi+1])):
            brut = my_forward_pass(clf,X_train[i])[-2]
            j=0
            for x in brut:
                X_train_meta[i][j] = 0.00 if x <= 0.33 else 0.5 if x < 0.67 else 1
                j=j+1

        # Récupération de la dernière couche du réseau et transformation des données de test
        for i in range(len(X_test)):
            brut = my_forward_pass(clf,X_test[i])[-2]
            j=0
            for x in brut:
                X_test_meta[i][j] = 0.00 if x <= 0.33 else 0.5 if x < 0.67 else 1
                j=j+1

        # Construction et évaluation d'un arbre de décision construit sur les données transformées
        decisionTree_meta.fit(X_train_meta,y_train_origine[0:nbi+1])
        nbRegles_meta = decisionTree_meta.get_n_leaves()
        DTscoreTest_meta = decisionTree_meta.score(X_test_meta, y_test_origine)
        print("Arbre de décision données transformées : Nb feuilles (= règles) : ",nbRegles_meta," Score sur les données d'apprentissage = ", decisionTree_meta.score(X_train_meta, y_train_origine[0:nbi+1])," Score sur les données de test = ", DTscoreTest_meta)
        
        out_data[s,inst_idx] = [nbRegles_origine,DTscoreTest_origine,nbRegles_meta,DTscoreTest_meta,NNscoreTest]
        inst_idx = inst_idx+1

out_data

Shuffle  0
Nb instances =  1000
Arbre de décision données d'origine : Nb feuilles (= règles) :  37  Score sur les données d'apprentissage =  1.0  Score sur les données de test =  0.953
Réseau de neurones : Score sur les données d'apprentissage =  1.0 Score sur les données de test =  0.97
Arbre de décision données transformées : Nb feuilles (= règles) :  4  Score sur les données d'apprentissage =  0.995004995004995  Score sur les données de test =  0.972
Nb instances =  2000
Arbre de décision données d'origine : Nb feuilles (= règles) :  57  Score sur les données d'apprentissage =  1.0  Score sur les données de test =  0.961
Réseau de neurones : Score sur les données d'apprentissage =  0.9895052473763118 Score sur les données de test =  0.972
Arbre de décision données transformées : Nb feuilles (= règles) :  3  Score sur les données d'apprentissage =  0.960519740129935  Score sur les données de test =  0.949
Nb instances =  3000
Arbre de décision données d'origine : Nb feuilles (= règle

C:\Users\ChristiandeSainteMar\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:500: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of f AND g EVALUATIONS EXCEEDS LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


Réseau de neurones : Score sur les données d'apprentissage =  0.9728610198543065 Score sur les données de test =  0.977
Arbre de décision données transformées : Nb feuilles (= règles) :  11  Score sur les données d'apprentissage =  0.9617197543208114  Score sur les données de test =  0.966
Nb instances =  8000
Arbre de décision données d'origine : Nb feuilles (= règles) :  148  Score sur les données d'apprentissage =  1.0  Score sur les données de test =  0.978
Réseau de neurones : Score sur les données d'apprentissage =  0.9993750781152356 Score sur les données de test =  0.996
Arbre de décision données transformées : Nb feuilles (= règles) :  3  Score sur les données d'apprentissage =  0.9900012498437696  Score sur les données de test =  0.99
Nb instances =  9000
Arbre de décision données d'origine : Nb feuilles (= règles) :  157  Score sur les données d'apprentissage =  1.0  Score sur les données de test =  0.975


C:\Users\ChristiandeSainteMar\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:500: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of f AND g EVALUATIONS EXCEEDS LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


Réseau de neurones : Score sur les données d'apprentissage =  0.998555716031552 Score sur les données de test =  0.994
Arbre de décision données transformées : Nb feuilles (= règles) :  9  Score sur les données d'apprentissage =  0.9918897900233308  Score sur les données de test =  0.988
Nb instances =  10000
Arbre de décision données d'origine : Nb feuilles (= règles) :  162  Score sur les données d'apprentissage =  1.0  Score sur les données de test =  0.981
Réseau de neurones : Score sur les données d'apprentissage =  0.9449 Score sur les données de test =  0.94
Arbre de décision données transformées : Nb feuilles (= règles) :  9  Score sur les données d'apprentissage =  0.9421  Score sur les données de test =  0.944
Shuffle  1
Nb instances =  1000
Arbre de décision données d'origine : Nb feuilles (= règles) :  43  Score sur les données d'apprentissage =  1.0  Score sur les données de test =  0.965
Réseau de neurones : Score sur les données d'apprentissage =  1.0 Score sur les donné

C:\Users\ChristiandeSainteMar\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:500: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of f AND g EVALUATIONS EXCEEDS LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


Réseau de neurones : Score sur les données d'apprentissage =  1.0 Score sur les données de test =  0.98
Arbre de décision données transformées : Nb feuilles (= règles) :  3  Score sur les données d'apprentissage =  0.9780109945027486  Score sur les données de test =  0.978
Nb instances =  3000
Arbre de décision données d'origine : Nb feuilles (= règles) :  75  Score sur les données d'apprentissage =  1.0  Score sur les données de test =  0.97
Réseau de neurones : Score sur les données d'apprentissage =  1.0 Score sur les données de test =  0.992
Arbre de décision données transformées : Nb feuilles (= règles) :  3  Score sur les données d'apprentissage =  0.9946684438520493  Score sur les données de test =  0.99
Nb instances =  4000
Arbre de décision données d'origine : Nb feuilles (= règles) :  101  Score sur les données d'apprentissage =  1.0  Score sur les données de test =  0.969


C:\Users\ChristiandeSainteMar\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:500: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of f AND g EVALUATIONS EXCEEDS LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


Réseau de neurones : Score sur les données d'apprentissage =  0.9385153711572107 Score sur les données de test =  0.934
Arbre de décision données transformées : Nb feuilles (= règles) :  10  Score sur les données d'apprentissage =  0.9042739315171208  Score sur les données de test =  0.893
Nb instances =  5000
Arbre de décision données d'origine : Nb feuilles (= règles) :  114  Score sur les données d'apprentissage =  1.0  Score sur les données de test =  0.969
Réseau de neurones : Score sur les données d'apprentissage =  0.9986002799440112 Score sur les données de test =  0.998
Arbre de décision données transformées : Nb feuilles (= règles) :  2  Score sur les données d'apprentissage =  0.9926014797040592  Score sur les données de test =  0.996
Nb instances =  6000
Arbre de décision données d'origine : Nb feuilles (= règles) :  124  Score sur les données d'apprentissage =  1.0  Score sur les données de test =  0.974
Réseau de neurones : Score sur les données d'apprentissage =  0.99950

In [ ]:
print("Shape of the results array: ",out_data.shape)

print("Dimensions of the results array: ",out_data.ndim)

shuffle_Means = out_data.mean(axis=0)
shuffle_Std = out_data.std(axis=0)
print("Mean of results array - Along axis Shuffle: ",shuffle_Means)
print("Standard deviation of results array - Along axis Shuffle: ",shuffle_Std)

#print("Mean of results array - Along axis Nb Instances: ",out_data.mean(axis=1))
#print("Mean of results array - Along axis Values : ",out_data.mean(axis=2))

In [ ]:
import matplotlib.pyplot as plt
fig, (ax0, ax1) = plt.subplots(1, 2, constrained_layout=True)#nrows=2)
#plt.title('Average number of rules with growing number of training instances')
ax0.errorbar(list(range(1000,10001,1000)), shuffle_Means[:,0], yerr=shuffle_Std[:,0], fmt='-o')
ax1.errorbar(list(range(1000,10001,1000)), shuffle_Means[:,2], yerr=shuffle_Std[:,2], fmt='-o')
ax0.set_title("(a)")
ax0.set_xlabel("Nb training instances")
ax0.set_ylabel("Nb rules")
ax1.set_title("(b)")
ax1.set_xlabel("Nb training instances")
#ax1.set_ylabel("Nb rules")
fig.set_size_inches(15,3)
plt.savefig("fig-5-5-3.png",format="png")